# INFO 2950 Data Science Final Project Appendix

By Vivian Chiang (vc342), Elena Limanjaya (eel56), and Mia Moon (mm2632)

In [2]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


import statsmodels.api as sm

import csv

# Data Cleaning: Creating the Primary Dataset 

## a. Obtaining Rotten Tomato Audience Scores

To start off with, we load the two primary datasets from datasets downloaded from Kaggle. Netflix_df1 contains the titles with rotten tomato scores from both audience and critics. Netflix_df2 contains more information regarding the characteristics regarding each title, such as title, directors, countries produced, etc.

Since both datasets contain data relating to movies and TV shows on Netflix, we used SQL to select only the movies. Due to the nature of our project, because we were only worried about audience satisfaction of the movies, we chose to only use the Rotten Tomato audience score to serve as our metric to evaluate how "good" a movie is. Therefore, we took out all of the movies that had NaNs in rotten tomato ratings for audience scores.

Finally, both datasets were merged via inner join, and we filtered out only the categories that we were interested in analyzing, especially since the two datasets had overlapping categories.

In [3]:
# read datasets
netflix_df1 = pd.read_csv("netflix_titles_enriched.csv")
netflix_df2 = pd.read_csv("netflix_titles.csv")

In [4]:
# data Cleaning
netflix_cleaned_df1 = %sql SELECT * FROM netflix_df1 WHERE type='Movie' AND isnan(rottentomatoes_audience_score)is not null \
AND isnan(rottentomatoes_tomatometer_score) is not null ORDER BY title ASC

combined_df = %sql SELECT * FROM netflix_cleaned_df1 INNER JOIN netflix_df2 ON netflix_cleaned_df1.title = netflix_df2.title \
ORDER BY netflix_cleaned_df1.title ASC

combined_df = %sql SELECT "cast", "title", "country", "release_year", "duration", "rating", \
"listed_in", "rottentomatoes_audience_score", "rottentomatoes_audience_#reviews" FROM combined_df

combined_df.head(3)

,cast,title,country,release_year,duration,rating,listed_in,rottentomatoes_audience_score,rottentomatoes_audience_#reviews
0,"Nesta Cooper, Kate Walsh, John Michael Higgins...",#realityhigh,United States,2017,99 min,TV-14,Comedies,51.0,175
1,"Steven Strait, Camilla Belle, Cliff Curtis, Jo...","10,000 B.C.","United States, South Africa",2008,109 min,PG-13,Action & Adventure,37.0,250000
2,"PJ McCabe, Brianne Moncrief, Sarah Baldwin, Ji...",13 Cameras,United States,2015,90 min,NR,"Horror Movies, Independent Movies, Thrillers",35.0,750


Since we want to use the audience ratings as our chosen y-variable for the basis of our comparison, we also made sure to convert the ratings into floats from strings. That way, we are able to quantify our results for the  regression models used later on.

In [5]:
# converting number of reviews into floats 
test = %sql SELECT "title", "rottentomatoes_audience_score" as "audience_score", "rottentomatoes_audience_#reviews" AS "num" FROM combined_df ORDER BY \
"rottentomatoes_audience_#reviews" DESC

for i in np.arange(test["num"].size-1):
    if "[" in test["num"][i]:
        test["num"][i] = int(5000)
    else:
        test["num"][i] = int(test["num"][i])
        
average_rating = test["audience_score"].mean()
print(average_rating)
test.head(5)

/var/folders/c_/px9zxk3s5vlcsqqvsswrdjx00000gn/T/ipykernel_3634/4254127083.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["num"][i] = int(5000)
/var/folders/c_/px9zxk3s5vlcsqqvsswrdjx00000gn/T/ipykernel_3634/4254127083.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["num"][i] = int(test["num"][i])


62.803013392857146


,title,audience_score,num
0,Adore,41.0,5000
1,Aftershock,24.0,5000
2,Ai Weiwei: Never Sorry,87.0,5000
3,Alice Doesn't Live Here Anymore,82.0,5000
4,American Honey,63.0,5000


One problem we then encountered from using audience score is that the number of reviews may vary per movie. Lets say a movie had a 10.0 rating but only 1 audience review versus a movie with a million reviews and a 7.0 review. It would be hard to gauge general audience satisfaction based on one audience’s opinion despite having a high rating. Therefore, our solution around this is to have a standardization score that allows us to compare the reviews without having to worry about the number of reviews. However, we ran into the issue of how we define movies with “little” movie reviews. According to IMDB, the minimum vote needed to be listed in Top 250 is 3000 votes [reference: http://www.imdb.com/chart/top]. Hence, we used that as a cutoff. However, a large limitation to be noted here is that this number comes from IMDB, which may have a different metric compared to Rotten Tomatoes. Thus, this is also something we have to take into account in our limitation analysis. 


In [6]:
# standardization of ratings
standardized_df = %sql SELECT "title", "audience_score", "num" FROM test WHERE num > 3000 ORDER BY num DESC 
standardized_df["num"] = standardized_df["num"].astype(int)
standardized_df["audience_score"] = (standardized_df["num"]/(standardized_df["num"]+3000)) \
* standardized_df["audience_score"] + (3000/(standardized_df["num"]+3000)) * average_rating
standardized_df

,title,audience_score,num
0,A Haunted House,51.453962,75000
1,A Serious Man,67.800116,75000
2,A Thousand Words,47.607808,75000
3,About Time,80.300116,75000
4,Act of Valor,71.646270,75000
...,...,...,...
769,Wadjda,84.312636,17500
770,"Waiting for ""Superman""",80.898002,17500
771,While We're Young,52.727270,17500
772,Wish I Was Here,59.556539,17500


In [7]:
# add standardized scores to the merged dataframe
cleaned_df = %sql SELECT combined_df.title, combined_df.country, combined_df.release_year, combined_df.duration, \
combined_df.listed_in, combined_df.rating, standardized_df.audience_score, standardized_df.num FROM combined_df \
INNER JOIN standardized_df ON combined_df['title'] = standardized_df['title']

cleaned_df.head(3)

,title,country,release_year,duration,listed_in,rating,audience_score,num
0,"10,000 B.C.","United States, South Africa",2008,109 min,Action & Adventure,PG-13,37.305965,250000
1,13 Sins,United States,2014,93 min,"Horror Movies, Thrillers",R,50.086575,7500
2,13TH,United States,2016,101 min,Documentaries,TV-MA,82.229432,7500


We then ensured each category was in the type (for example, a quantitative variable should be an int or float) for our analysis. Here, we notice that duration is still in string form. 


In [12]:
#checking the types to ensure they are optimal
cleaned_df.dtypes

title              object
country            object
release_year        int64
duration           object
listed_in          object
rating             object
audience_score    float64
num                 int64
dtype: object

In [7]:
# change the duration into a float 

cleaned_df['duration'] = cleaned_df['duration'].astype("string")
cleaned_df['duration'] = cleaned_df['duration'].str.replace("min", "")
cleaned_df['duration']

cleaned_df['duration'] = pd.to_numeric(cleaned_df["duration"])
cleaned_df.head(3)

,title,country,release_year,duration,listed_in,rating,audience_score,num
0,"10,000 B.C.","United States, South Africa",2008,109,Action & Adventure,PG-13,37.305965,250000
1,13 Sins,United States,2014,93,"Horror Movies, Thrillers",R,50.086575,7500
2,13TH,United States,2016,101,Documentaries,TV-MA,82.229432,7500


## b. Obtaining Information on Revenue and Budget 

Once we cleaned and merged the first two data frames containing information about the Rotten Tomato audience scores, as stated in our concerns during Phase 4, we were initially working with many categorical variables (ex. genre, director), each with multiple overlapping categories (ex. Within the genre, there are comedies, thrillers, dramas, etc.). After consulting with various INFO 2950 staff, we decided to seek two additional datasets for more quantitative variables to supplement our analysis. In particular, we chose datasets that had more information about revenue and budget. We used SQL to inner join this first dataset with our dataset containing rotten tomato audience scores. We ensured that all duplicates and NaNs were dropped. 

In [15]:
# budget/revenue dataset 1
movies_df = pd.read_csv("movies.csv")
new_combined_df =%sql SELECT cleaned_df.title, cleaned_df.release_year, cleaned_df.audience_score, cleaned_df.rating, cleaned_df.listed_in, cleaned_df.country,cleaned_df.duration,\
movies_df.budget, movies_df.gross AS revenue FROM cleaned_df INNER JOIN movies_df ON cleaned_df.title = movies_df.name
new_combined_df = new_combined_df.dropna()
new_combined_df = new_combined_df.drop_duplicates()
new_combined_df
new_combined_df.sort_values(by=['title'])



,title,release_year,audience_score,rating,listed_in,country,duration,budget,revenue
215,16 Blocks,2006,57.097804,PG-13,Action & Adventure,"United States, Germany",102 min,52000000.0,65664721.0
290,17 Again,2009,66.950233,PG-13,Comedies,United States,102 min,20000000.0,136316880.0
472,20th Century Women,2016,72.361417,R,"Dramas, Independent Movies",United States,119 min,7000000.0,7214806.0
118,28 Days,2000,50.151814,PG-13,"Comedies, Dramas",United States,104 min,43000000.0,62198945.0
423,3 Days to Kill,2014,44.697401,PG-13,Action & Adventure,"United States, France, Serbia",117 min,28000000.0,53260230.0
...,...,...,...,...,...,...,...,...,...
268,You Don't Mess with the Zohan,2008,45.211103,UR,"Action & Adventure, Comedies",United States,113 min,90000000.0,204313400.0
345,Young Adult,2011,50.183115,R,"Comedies, Dramas, Independent Movies",United States,94 min,12000000.0,22939027.0
225,Zodiac,2007,76.831656,R,"Cult Movies, Dramas, Thrillers",United States,158 min,65000000.0,84785914.0
284,Zombieland,2009,85.724937,R,"Comedies, Horror Movies",United States,88 min,23600000.0,102392080.0


The same process was repeated for the second dataset. 

In [17]:
# budget/revenue dataset 2
movies_df2 = pd.read_csv("movie_dataset.csv")

new_combined_df2 =%sql SELECT cleaned_df.title, cleaned_df.release_year, cleaned_df.audience_score, cleaned_df.rating, cleaned_df.listed_in, cleaned_df.country,cleaned_df.duration,\
movies_df2.budget, movies_df2.revenue FROM cleaned_df INNER JOIN movies_df2 ON \
cleaned_df.title = movies_df2.original_title WHERE budget != 0 AND revenue != 0
new_combined_df2 = new_combined_df2.dropna()
new_combined_df2.sort_values(by=['title'])


,title,release_year,audience_score,rating,listed_in,country,duration,budget,revenue
142,16 Blocks,2006,57.097804,PG-13,Action & Adventure,"United States, Germany",102 min,55000000,65664721
147,17 Again,2009,66.950233,PG-13,Comedies,United States,102 min,20000000,136267476
182,3 Days to Kill,2014,44.697401,PG-13,Action & Adventure,"United States, France, Serbia",117 min,28000000,52597999
181,30 Minutes or Less,2011,41.954544,R,"Action & Adventure, Comedies",United States,83 min,28000000,40547440
70,50 First Dates,2004,64.973949,PG-13,"Comedies, Romantic Movies",United States,99 min,75000000,196482882
...,...,...,...,...,...,...,...,...,...
54,You Don't Mess with the Zohan,2008,45.211103,UR,"Action & Adventure, Comedies",United States,113 min,90000000,201596308
265,Young Adult,2011,50.183115,R,"Comedies, Dramas, Independent Movies",United States,94 min,12000000,22939027
61,Zodiac,2007,76.831656,R,"Cult Movies, Dramas, Thrillers",United States,158 min,65000000,84785914
203,Zombieland,2009,85.724937,R,"Comedies, Horror Movies",United States,88 min,23600000,102391382


We created a final data frame that concatenated titles from both datasets, having a total of **466** movies to work with. This final dataframe is then converted into a csv file ready for analysis. 


In [18]:
# create final data dataframe
data_df = pd.concat([new_combined_df, new_combined_df2]).drop_duplicates()



data_df=data_df.sort_values(by=['budget'], ascending=False).drop_duplicates(['title'])
data_df=data_df.sort_values(by=['title'], ascending=False).drop_duplicates(['revenue'])

data_df = data_df.sort_values(by=['title'])
data_df


,title,release_year,audience_score,rating,listed_in,country,duration,budget,revenue
142,16 Blocks,2006,57.097804,PG-13,Action & Adventure,"United States, Germany",102 min,55000000.0,65664721.0
290,17 Again,2009,66.950233,PG-13,Comedies,United States,102 min,20000000.0,136316880.0
472,20th Century Women,2016,72.361417,R,"Dramas, Independent Movies",United States,119 min,7000000.0,7214806.0
118,28 Days,2000,50.151814,PG-13,"Comedies, Dramas",United States,104 min,43000000.0,62198945.0
423,3 Days to Kill,2014,44.697401,PG-13,Action & Adventure,"United States, France, Serbia",117 min,28000000.0,53260230.0
...,...,...,...,...,...,...,...,...,...
54,You Don't Mess with the Zohan,2008,45.211103,UR,"Action & Adventure, Comedies",United States,113 min,90000000.0,201596308.0
345,Young Adult,2011,50.183115,R,"Comedies, Dramas, Independent Movies",United States,94 min,12000000.0,22939027.0
225,Zodiac,2007,76.831656,R,"Cult Movies, Dramas, Thrillers",United States,158 min,65000000.0,84785914.0
284,Zombieland,2009,85.724937,R,"Comedies, Horror Movies",United States,88 min,23600000.0,102392080.0


In [30]:
data_df.to_csv('final_project_movies.csv')

# Data Cleaning: Converting Categorical Variables to Numerical Rankings



From our initial dataset, we had two major categorical variables we wanted to incorporate into our analysis: Motion Picture Association ratings and production country. We decided the easiest way to incorporate these two variables into a multivariate regression analysis is to convert their values into numerical ranked values. In particular, to fit the context of our project, the dataset used is a subset that focuses on comedy movies. 


## a. Motion Picture Association Rating Ranking 

The MA ratings were ranked based on “maturity.” The higher the number, the more mature the audience needed to be viewing the movie. 


In [65]:
# isolating only comedy movies 

data_df[['first_genre','second_genre','third_genre']] = data_df.listed_in.str.split(',',expand=True)

data_df['first_genre'] = data_df['first_genre'].str.strip()
data_df['second_genre'] = data_df['second_genre'].str.strip()
data_df['third_genre'] = data_df['third_genre'].str.strip()


genre_df2 = %sql SELECT * FROM data_df
genre_df2 = genre_df2.melt(id_vars = ["title", "audience_score",  "release_year", "duration", "rating", "budget", "revenue", "country"],  value_vars = ["first_genre", "second_genre", "third_genre"], value_name = "genre")

genre_df2 =  genre_df2.dropna()

comedies_df = genre_df2[genre_df2["genre"] == "Comedies"]
comedies_df = comedies_df.iloc[:,:10]


# change MA ratings to numerical 

comedies_df= comedies_df.rename(columns={'rating': 'MA_rating'})

pd.value_counts(comedies_df['MA_rating'])
rating_map = {"G": 1,  "PG": 2, "TV-PG" :3, "PG-13": 4, "TV-MA":5, "R": 6}


comedies_df['MA_rating_rank'] = comedies_df['MA_rating'].map(rating_map)


comedies_df

,title,audience_score,release_year,duration,MA_rating,budget,revenue,country,variable,genre,MA_rating_rank
1,17 Again,66.950233,2009,102,PG-13,20000000.0,136316880.0,United States,first_genre,Comedies,4.0
3,28 Days,50.151814,2000,104,PG-13,43000000.0,62198945.0,United States,first_genre,Comedies,4.0
6,50 First Dates,64.973949,2004,99,PG-13,75000000.0,198520934.0,United States,first_genre,Comedies,4.0
7,A Bad Moms Christmas,46.751660,2017,104,R,28000000.0,130560428.0,"United States, China",first_genre,Comedies,6.0
10,A Haunted House,51.453962,2013,86,R,2500000.0,60141683.0,United States,first_genre,Comedies,6.0
...,...,...,...,...,...,...,...,...,...,...,...
921,Wild Wild West,28.412684,1999,106,PG-13,170000000.0,222104681.0,United States,second_genre,Comedies,4.0
927,You Don't Mess with the Zohan,45.211103,2008,113,UR,90000000.0,201596308.0,United States,second_genre,Comedies,NaN
931,Zoom,37.361417,2006,88,PG,35000000.0,12506362.0,United States,second_genre,Comedies,2.0
1163,Monty Python and the Holy Grail,94.618218,1975,92,PG,400000.0,5028948.0,United Kingdom,third_genre,Comedies,2.0


## b. Country Ranking

For countries, because a movie can be produced in multiple countries, we needed to separate the countries into individual columns before melting into one country column. 


In [66]:
#creating a country dataframe that breaks down each movie's country 

country_length = comedies_df['country'].str.count(",")
country_max = np.where(country_length == country_length.max())[0]

data_df2 = comedies_df.copy()

data_df2[['first_country','second_country','third_country','fourth_country','fifth_country',]] = comedies_df.country.str.split(', ',expand=True)

# country_df = %sql SELECT title, first_country, second_country, third_country, fourth_country, fifth_country FROM data_df2
country_df = data_df2.melt(id_vars = "title", value_vars = ['first_country','second_country','third_country','fourth_country',\
                                                              'fifth_country'])
country_df = country_df.dropna()
country_df = %sql SELECT data_df2.title, audience_score, release_year, duration, budget, revenue, genre, country_df.value AS country FROM data_df2 LEFT JOIN country_df ON country_df.title = data_df2.title
country_df

,title,audience_score,release_year,duration,budget,revenue,genre,country
0,17 Again,66.950233,2009,102,20000000.0,136316880.0,Comedies,United States
1,28 Days,50.151814,2000,104,43000000.0,62198945.0,Comedies,United States
2,50 First Dates,64.973949,2004,99,75000000.0,198520934.0,Comedies,United States
3,A Bad Moms Christmas,46.751660,2017,104,28000000.0,130560428.0,Comedies,China
4,A Haunted House,51.453962,2013,86,2500000.0,60141683.0,Comedies,United States
...,...,...,...,...,...,...,...,...
224,Maps to the Stars,46.751660,2014,112,15000000.0,4510934.0,Comedies,Canada
225,The Death of Stalin,72.301130,2017,107,13000000.0,24646055.0,Comedies,France
226,Scott Pilgrim vs. the World,83.642747,2010,112,60000000.0,48917974.0,Comedies,United States
227,The Smurfs 2,56.261654,2013,105,105000000.0,347545360.0,Comedies,United States


In [67]:
# check all movies is comedy
country_df['genre'].drop_duplicates().array

<PandasArray>
['Comedies']
Length: 1, dtype: object

For the country rankings, we incorporated another dataset that contained information regarding the number of movies produced from that country. Thus, we chose to use that number to represent each country. For example, according to the dataset, the United States has produced 22818 movies, which will be the value in the linear regression model. 

In [68]:
# rank countries in order of number of movies made
country_ranking_df = pd.read_csv("countries_ranking.csv", thousands=',')
cr_df = country_ranking_df.iloc[:, :2]
cr_df

,production_countries,number
0,United States,22818
1,United Kingdom,4064
2,China,2346
3,France,3735
4,Japan,1582
...,...,...
176,Oman,1
177,Honduras,1
178,Belize,1
179,Kiribati,1


In [69]:
# # find list of countries in dataset
country_array = country_df['country'].drop_duplicates().array
country_array

<PandasArray>
[       'United States',                'China',               'France',
       'United Kingdom',              'Germany',               'Canada',
                'Italy',               'Mexico',            'Australia',
               'Sweden',            'Hong Kong',                'Japan',
 'United Arab Emirates',       'Czech Republic',              'Belgium',
              'Finland',                'Spain']
Length: 17, dtype: object

In [70]:
# filter number of movies per country data by countries in the dataset
cr_df = cr_df[cr_df.production_countries.isin(country_array)]
cr_df

,production_countries,number
0,United States,22818
1,United Kingdom,4064
2,China,2346
3,France,3735
4,Japan,1582
5,Germany,1729
7,Canada,1748
8,Australia,911
11,Hong Kong,423
12,Italy,1642


In [71]:
# create dictionary for country to ranking number
country_map = dict([(i,x) for i,x in zip(cr_df['production_countries'], cr_df['number'])])

# change country to ranking number
ranked_country_df = country_df
ranked_country_df['country_rank'] = ranked_country_df['country'].map(country_map)
ranked_country_df= ranked_country_df.rename(columns={'country': 'country_name'})

ranked_country_df

,title,audience_score,release_year,duration,budget,revenue,genre,country_name,country_rank
0,17 Again,66.950233,2009,102,20000000.0,136316880.0,Comedies,United States,22818
1,28 Days,50.151814,2000,104,43000000.0,62198945.0,Comedies,United States,22818
2,50 First Dates,64.973949,2004,99,75000000.0,198520934.0,Comedies,United States,22818
3,A Bad Moms Christmas,46.751660,2017,104,28000000.0,130560428.0,Comedies,China,2346
4,A Haunted House,51.453962,2013,86,2500000.0,60141683.0,Comedies,United States,22818
...,...,...,...,...,...,...,...,...,...
224,Maps to the Stars,46.751660,2014,112,15000000.0,4510934.0,Comedies,Canada,1748
225,The Death of Stalin,72.301130,2017,107,13000000.0,24646055.0,Comedies,France,3735
226,Scott Pilgrim vs. the World,83.642747,2010,112,60000000.0,48917974.0,Comedies,United States,22818
227,The Smurfs 2,56.261654,2013,105,105000000.0,347545360.0,Comedies,United States,22818


Finally, the created ranked_country_df was merged with the country_comedies_df via SQL inner join. This created a total of 226 rows of data points that are then converted into a csv file ready for individual and multivariable regression analysis. 


In [73]:
#inner join the country_df with the dataframe with the standardized reviews 
country_comedies = %sql SELECT comedies_df.* , ranked_country_df.* FROM comedies_df INNER JOIN ranked_country_df ON \
comedies_df.title = ranked_country_df.title
country_comedies = country_comedies.dropna()
country_comedies = country_comedies.loc[:,~country_comedies.columns.duplicated()].copy()


country_comedies

,title,audience_score,release_year,duration,MA_rating,budget,revenue,country,variable,genre,MA_rating_rank,country_name,country_rank
0,17 Again,66.950233,2009,102,PG-13,20000000.0,136316880.0,United States,first_genre,Comedies,4.0,United States,22818
1,28 Days,50.151814,2000,104,PG-13,43000000.0,62198945.0,United States,first_genre,Comedies,4.0,United States,22818
2,50 First Dates,64.973949,2004,99,PG-13,75000000.0,198520934.0,United States,first_genre,Comedies,4.0,United States,22818
3,A Bad Moms Christmas,46.751660,2017,104,R,28000000.0,130560428.0,"United States, China",first_genre,Comedies,6.0,China,2346
4,A Haunted House,51.453962,2013,86,R,2500000.0,60141683.0,United States,first_genre,Comedies,6.0,United States,22818
...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,Maps to the Stars,46.751660,2014,112,R,15000000.0,4510934.0,"Canada, Germany, France, United States",first_genre,Comedies,6.0,Canada,1748
225,The Death of Stalin,72.301130,2017,107,R,13000000.0,24646055.0,"United Kingdom, France, Belgium, Canada, Unite...",first_genre,Comedies,6.0,France,3735
226,Scott Pilgrim vs. the World,83.642747,2010,112,PG-13,60000000.0,48917974.0,"United States, United Kingdom, Canada, Japan",second_genre,Comedies,4.0,United States,22818
227,The Smurfs 2,56.261654,2013,105,PG,105000000.0,347545360.0,"United States, France, Canada, Belgium",second_genre,Comedies,2.0,United States,22818


In [74]:
country_comedies.to_csv('comedies_country_rank.csv')